Readme:
- Click copy and edit
- Select GPU accelerator
- Click run all

In [ ]:
#Loading of important libraries that are used throughout 
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O
import cv2 # computer vision library
import tensorflow as tf # machine learning library
import keras # Python interface to tensorflow
import matplotlib.pyplot as plt # data visualization tool
from tensorflow.python.keras import backend as K #to utilize more of keras' functionality


In [ ]:
#path to the training and test set
train_dir='/kaggle/input/siim-isic-melanoma-classification/jpeg/train/'
test_dir = '/kaggle/input/siim-isic-melanoma-classification/jpeg/test/'
#loading the training and test set
train=pd.read_csv('/kaggle/input/siim-isic-melanoma-classification/train.csv')
test=pd.read_csv('/kaggle/input/siim-isic-melanoma-classification/test.csv')

In [ ]:
#shows the five first rows of the dataframe
train.head()

In [ ]:
#include the path to the images in the dataframe 
train['path'] = train_dir + train.image_name + ".jpg"
train.head()
test['path'] = test_dir + test.image_name + ".jpg"

In [ ]:
#there are some duplicate images in the training data, these images might adversely impact our model, 
# so, lets remove these images
dup = pd.read_csv("/kaggle/input/siim-list-of-duplicates/2020_Challenge_duplicates.csv")

drop_idx_list = []
for dup_image in dup.ISIC_id_paired:
    for idx,image in enumerate(train.image_name):
        if image == dup_image:
            drop_idx_list.append(idx)

print("no. of duplicates in training dataset:",len(drop_idx_list))

train.drop(drop_idx_list,inplace=True)

print("updated dimensions of the training dataset:",train.shape)


In [ ]:
#show one instance of the images that we have:
img=cv2.imread('../input/siim-isic-melanoma-classification/jpeg/train/ISIC_0015719.jpg')   
img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
plt.imshow(img)


In [ ]:
#show how many of the different image types we have, we can see that we have many more benign images than malignant
train.target.value_counts()

In [ ]:
#to have a more balanced dataset, we create a new dataframe that contains a more equal percentage of each type of target image
df_0=train[train['target']==0].sample(3000)
df_1=train[train['target']==1]
train=pd.concat([df_0,df_1])
train=train.reset_index()

In [ ]:
train.shape

In [ ]:
train.head()

In [ ]:
# we will resize the given images to 128 x 128 size images for faster processing
IMG_DIM = (128, 128)

In [ ]:
# Keras provides some amazing libraries to work with images, lets import them
from keras.preprocessing.image import ImageDataGenerator, load_img, img_to_array, array_to_img

In [ ]:
# We will reserve 20% of our training data for the validation purpose
from sklearn.model_selection import train_test_split
X_train, X_val, y_train, y_val = train_test_split(train, train.target, test_size=0.2, random_state=42)

In [ ]:

train_files = X_train.path
val_files = X_val.path

# load images using load_img function from keras preprocessing 
# target_size is used to load the images with smaller size
# img_to_array will tranform the loaded image to an array
train_imgs = [img_to_array(load_img(img, target_size=IMG_DIM)) for img in train_files]
validation_imgs = [img_to_array(load_img(img, target_size=IMG_DIM)) for img in val_files]

# convert the list of arrays to array
train_imgs = np.array(train_imgs)
train_labels = y_train

validation_imgs = np.array(validation_imgs)
val_labels = y_val


print('Train dataset shape:', train_imgs.shape, 
      '\tValidation dataset shape:', validation_imgs.shape)

In [ ]:
train_imgs_scaled = train_imgs.astype('float32')
validation_imgs_scaled  = validation_imgs.astype('float32')

# divide the pixels by 255 to scale the pixels between 0 and 1
train_imgs_scaled /= 255
validation_imgs_scaled /= 255

print(train_imgs[0].shape)

# array_to_img function will convert the given array to image
array_to_img(train_imgs[0])

In [ ]:
# setup basic configuration
batch_size = 100
num_classes = 2
epochs = 100
input_shape = (128, 128, 3)

In [ ]:
# focal loss, because we have an imbalanced data set
def focal_loss(alpha=0.25,gamma=2.0):
    def focal_crossentropy(y_true, y_pred):
        bce = K.binary_crossentropy(y_true, y_pred)
        
        y_pred = K.clip(y_pred, K.epsilon(), 1.- K.epsilon())
        p_t = (y_true*y_pred) + ((1-y_true)*(1-y_pred))
        
        alpha_factor = 1
        modulating_factor = 1

        alpha_factor = y_true*alpha + ((1-alpha)*(1-y_true))
        modulating_factor = K.pow((1-p_t), gamma)

        # compute the final loss and return
        return K.mean(alpha_factor*modulating_factor*bce, axis=-1)
    return focal_crossentropy

In [ ]:
# Here we will import the necessary libraries
from keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout

# we will import sequential model and add different layers to it
from keras.models import Sequential

# import optimizers, please go through online tutorials if you want to learn what is the purpose of an optimizer
from keras import optimizers


# creating and instance of Sequential, which CNN is.
model = Sequential()

# add Conv2D layer(this is the convolutional layer we discussed earlier),filter size,kernel size,activation and padding are the parameters used
# This layer would create feature maps for each and every filter used
# feature maps created here are then taken through an activation function(relu here), which decides whether a certain feature is present 
# at a given location in the image.
model.add(Conv2D(16, kernel_size=(3, 3), activation='relu', 
                 input_shape=input_shape))
# Pooling layer used here will select the largest values on the feature maps and use these as inputs to subsequent layers
model.add(MaxPooling2D(pool_size=(2, 2)))


# another set of Convolutional & Max Pooling layers
model.add(Conv2D(64, kernel_size=(3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Conv2D(128, kernel_size=(3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Flatten())
# Finally the Dense Layer
model.add(Dense(512, activation='relu'))
# sigmoid function here will help us in performing binary classification
model.add(Dense(1, activation='sigmoid'))

model.compile(loss=focal_loss(),
              optimizer=optimizers.Adam(),
              metrics=[tf.keras.metrics.BinaryAccuracy(), tf.keras.metrics.FalsePositives(),tf.keras.metrics.FalseNegatives(),tf.keras.metrics.TrueNegatives(),tf.keras.metrics.TruePositives()])

model.summary()


In [ ]:
history = model.fit(x=train_imgs_scaled, y=train_labels,
                    validation_data=(validation_imgs_scaled, val_labels),
                    batch_size=batch_size,
                    epochs=epochs,
                    verbose=1)

In [ ]:
#checking model performance
f, (ax1, ax2) = plt.subplots(1, 2, figsize=(12, 4))
t = f.suptitle('Basic CNN Performance', fontsize=12)
f.subplots_adjust(top=0.85, wspace=0.3)

r = len(history.history['binary_accuracy']) +1
epoch_list = list(range(1,r))
ax1.plot(epoch_list, history.history['binary_accuracy'], label='Train Accuracy')
ax1.plot(epoch_list, history.history['val_binary_accuracy'], label='Validation Accuracy')
ax1.set_xticks(np.arange(0, 100, 5))
ax1.set_ylabel('Accuracy Value')
ax1.set_xlabel('Epoch')
ax1.set_title('Accuracy')
l1 = ax1.legend(loc="best")

ax2.plot(epoch_list, history.history['loss'], label='Train Loss')
ax2.plot(epoch_list, history.history['val_loss'], label='Validation Loss')
ax2.set_xticks(np.arange(0, 100, 5))
ax2.set_ylabel('Loss Value')
ax2.set_xlabel('Epoch')
ax2.set_title('Loss')
l2 = ax2.legend(loc="best")

In [ ]:
#the second model will also implement Dropout and Data Augmentation as measures to improve the accuracy.
from keras.preprocessing.image import ImageDataGenerator

#the ImageDataGenerator will rescale the images, zoom in, rotate, shift and flip the images to create a more diverste training set.
train_datagen = ImageDataGenerator(rescale=1./255, zoom_range=0.3, rotation_range=50,
                                   width_shift_range=0.2, height_shift_range=0.2, shear_range=0.2, 
                                   horizontal_flip=True, fill_mode='nearest')

val_datagen = ImageDataGenerator(rescale=1./255)

In [ ]:
#Under we can see how the ImageDataGenerator augments the images so that we get a more diversified training data.
#This will result in a more robust model.
img_id = 1

img_generator = train_datagen.flow(train_imgs[img_id:img_id+1], train_labels[img_id:img_id+1],
                                   batch_size=1)

img = [next(img_generator) for i in range(0,5)]

fig, ax = plt.subplots(1,5, figsize=(16, 6))
print('Labels:', [item[1][0] for item in img])
l = [ax[i].imshow(img[i][0][0]) for i in range(0,5)]

In [ ]:
#We now implement the same model as last time, only with the new augmented images as well as two dropout layers.
from tensorflow.keras.models import Sequential, save_model, load_model
from keras.callbacks import ModelCheckpoint

train_generator = train_datagen.flow(train_imgs, train_labels, batch_size=batch_size)
val_generator = val_datagen.flow(validation_imgs, val_labels, batch_size=batch_size)

input_shape = input_shape

model = tf.keras.Sequential() # define your model normally
model.add(Conv2D(16, kernel_size=(3, 3), activation='relu', 
                 input_shape=input_shape))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(64, kernel_size=(3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(128, kernel_size=(3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(128, kernel_size=(3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Flatten())
model.add(Dense(512, activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(512, activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(1, activation='sigmoid'))
model.compile(loss=focal_loss(),
              optimizer=optimizers.Adam(), 
              metrics=[tf.keras.metrics.BinaryAccuracy(), tf.keras.metrics.FalsePositives(),tf.keras.metrics.FalseNegatives(),tf.keras.metrics.TrueNegatives(),tf.keras.metrics.TruePositives()])


#we want to save the best model for our test predictions
#checkpointer = ModelCheckpoint(filepath="weights.hdf5", verbose=1, save_best_only=True)

history = model.fit_generator(train_generator, steps_per_epoch=train_imgs.shape[0]//100, epochs=100,
                              validation_data=val_generator, validation_steps=validation_imgs.shape[0]//100, 
                              verbose=1)

In [ ]:
f, (ax1, ax2) = plt.subplots(1, 2, figsize=(12, 4))
t = f.suptitle('CNN with Regularization & Augmentation', fontsize=12)
f.subplots_adjust(top=0.85, wspace=0.3)

r = len(history.history['binary_accuracy']) +1
epoch_list = list(range(1,r))
ax1.plot(epoch_list, history.history['binary_accuracy'], label='Train Accuracy')
ax1.plot(epoch_list, history.history['val_binary_accuracy'], label='Validation Accuracy')
ax1.set_xticks(np.arange(0, 101, 5))
ax1.set_ylabel('Accuracy Value')
ax1.set_xlabel('Epoch')
ax1.set_title('Accuracy')
l1 = ax1.legend(loc="best")

ax2.plot(epoch_list, history.history['loss'], label='Train Loss')
ax2.plot(epoch_list, history.history['val_loss'], label='Validation Loss')
ax2.set_xticks(np.arange(0, 101, 5))
ax2.set_ylabel('Loss Value')
ax2.set_xlabel('Epoch')
ax2.set_title('Loss')
l2 = ax2.legend(loc="best")

In [ ]:
# define parameters for transfer learning model training
batch_size = 32 # the total number of images processed per iteration
num_classes = 2 # we have two classes; benign and malignant
epochs = 100 # the number of iteration over the entire training set
input_shape = (128, 128, 3)

In [ ]:
# import optimizers from keras
from keras.optimizers import Adam, SGD, RMSprop

# use Adam optimizer
opt = Adam(lr=1e-5)

In [ ]:
train_generator = train_datagen.flow(train_imgs, y_train, batch_size=batch_size)
val_generator = val_datagen.flow(validation_imgs, y_val, batch_size=batch_size)

In [ ]:
from keras.models import Model
from keras.applications import vgg16

# initializing the VGG16 model with pre-trained weights which was trained on ImageNet. 
vgg = vgg16.VGG16(include_top=False, weights='imagenet', 
                                     input_shape=input_shape)

# flatten the output layer
output = vgg.layers[-1].output
output = keras.layers.Flatten()(output)
vgg_model = Model(vgg.input, output)

# set all layers to not be trained
vgg_model.trainable = False
for layer in vgg_model.layers:
    layer.trainable = False
    
pd.set_option('max_colwidth', -1)
layers = [(layer, layer.name, layer.trainable) for layer in vgg_model.layers]
pd.DataFrame(layers, columns=['Layer Type', 'Layer Name', 'Layer Trainable']) 

vgg_model.summary()

In [ ]:
# train the convolution layers from block4_conv1 to output layer in the model
vgg_model.trainable = True

set_trainable = False
for layer in vgg_model.layers:
    if layer.name in ['block5_conv1', 'block4_conv1']:
        set_trainable = True
    if set_trainable:
        layer.trainable = True
    else:
        layer.trainable = False
        
layers = [(layer, layer.name, layer.trainable) for layer in vgg_model.layers]
pd.DataFrame(layers, columns=['Layer Type', 'Layer Name', 'Layer Trainable'])  

In [ ]:
from keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout, InputLayer
from keras.models import Sequential
import tensorflow as tf
from tensorflow.keras.models import Sequential, save_model, load_model
from keras.callbacks import ModelCheckpoint

# creating an instance of Sequential model
model = Sequential()

# add the VGG16 model
model.add(vgg_model)

# add dense and dropout layers
model.add(Dense(512, activation='relu', input_dim=input_shape))
model.add(Dropout(0.4))
model.add(Dense(512, activation='relu'))
model.add(Dropout(0.4))
model.add(Dense(1, activation='sigmoid'))

# compiling the model
model.compile(loss=focal_loss(), metrics=[tf.keras.metrics.BinaryAccuracy(),tf.keras.metrics.FalsePositives(),tf.keras.metrics.FalseNegatives(),tf.keras.metrics.TrueNegatives(),tf.keras.metrics.TruePositives()],optimizer=optimizers.Adam(lr=1e-5))

#we want to save the best model for our test predictions
#checkpointer = ModelCheckpoint(filepath="weights.hdf5", verbose=1, save_best_only=True)

model.summary()

In [ ]:
#total number of iterations is always equal to the total number of training samples divided by the batch_size.
nb_train_steps = train_imgs.shape[0]//batch_size
nb_val_steps=validation_imgs.shape[0]//batch_size

print("Number of training and validation steps: {} and {}".format(nb_train_steps,nb_val_steps))

In [ ]:
# training of the model
history = model.fit_generator(train_generator, steps_per_epoch=nb_train_steps, epochs=epochs,
                           validation_data=val_generator, validation_steps=nb_val_steps, 
                              verbose=1)

In [ ]:
#checking model performance
f, (ax1, ax2) = plt.subplots(1, 2, figsize=(12, 4))
t = f.suptitle('VGG16 Performance', fontsize=12)
f.subplots_adjust(top=0.85, wspace=0.3)

epoch_list = list(range(1,101))
ax1.plot(epoch_list, history.history['binary_accuracy'], label='Train Accuracy')
ax1.plot(epoch_list, history.history['val_binary_accuracy'], label='Validation Accuracy')
ax1.set_xticks(np.arange(0, 101, 5))
ax1.set_ylabel('Accuracy Value')
ax1.set_xlabel('Epoch')
ax1.set_title('Accuracy')
l1 = ax1.legend(loc="best")

ax2.plot(epoch_list, history.history['loss'], label='Train Loss')
ax2.plot(epoch_list, history.history['val_loss'], label='Validation Loss')
ax2.set_xticks(np.arange(0, 101, 5))
ax2.set_ylabel('Loss Value')
ax2.set_xlabel('Epoch')
ax2.set_title('Loss')
l2 = ax2.legend(loc="best")